In [ ]:
!pip install efficientnet
!pip install cleanlab

In [ ]:
import matplotlib.pyplot as plt
import copy
import numpy as np
from tensorflow.keras.datasets import cifar10, cifar100
import tensorflow as tf
import numpy as np
from PIL import Image

In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
import efficientnet.keras as enet


# Swish defination
from keras.backend import sigmoid

class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))


from keras.layers import Activation
tf.keras.saving.get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# # Select a random image from the training set
# image_index = 2
# image = train_images[image_index]

# # Display the image
# plt.imshow(image)
# plt.show()


# Gnerating OOS images

## Adding noise to freqquency domain

In [ ]:
# import numpy as np
# import cv2

# # load the image
# img = image

# # apply FFT to the image
# f = np.fft.fft2(img)

# # shift the zero frequency component to the center of the spectrum
# fshift = np.fft.fftshift(f) *4

# # add bias to the zero frequency component
# rows, cols,_ = img.shape
# crow, ccol = rows//2, cols//2

# fshift[crow, ccol] = fshift[crow, ccol]   # add a bias of 1000

# # shift the zero frequency component back to the corner of the spectrum
# f_ishift = np.fft.ifftshift(fshift)

# # apply inverse FFT to get the image back
# img_back = np.fft.ifft2(f_ishift)
# img_back = np.abs(img_back)  # take the absolute value

# # convert the image back to 8-bit unsigned integer format
# img_back = np.uint8(img_back)

# # show the original and processed images
# plt.imshow(image)
# plt.show()
# plt.imshow(img_back)
# plt.show()



## Training AE oncifar 10 for oos generation

*   List item
*   List item



In [ ]:
from keras.datasets import cifar10
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(x_train, _), (x_test, _) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Define the autoencoder architecture
input_img = Input(shape=(32, 32, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same', name='encoder_output')(x)

# Decoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Create the autoencoder model
autoencoder = Model(input_img, decoded)

# Create a model to output the encoder layer
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoder_output').output)

# Create a model to output the decoder layer
decoder_input = Input(shape=(8, 8, 16), name='decoder_input')
x = autoencoder.layers[-5](decoder_input)
x = autoencoder.layers[-4](x)
x = autoencoder.layers[-3](x)
x = autoencoder.layers[-2](x)
x = autoencoder.layers[-1](x)
decoder_output = x
decoder = Model(inputs=decoder_input, outputs=decoder_output, name='decoder')

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
# Train the autoencoder
autoencoder.fit(x_train, x_train, epochs=10, batch_size=256, shuffle=True, validation_data=(x_test, x_test))

In [ ]:
encoder.save('/content/drive/MyDrive/ECE720/CIFAR_AE/encoder')
decoder.save('/content/drive/MyDrive/ECE720/CIFAR_AE/decoder')

# Main Step

Mixup images

In [ ]:
def shuffle_image_pieces(im1, im2, im3, im4):
    # Convert the images to NumPy arrays
    arr1 = np.array(im1)
    arr2 = np.array(im2)
    arr3 = np.array(im3)
    arr4 = np.array(im4)

    # Split each image into four pieces
    h, w, _ = arr1.shape
    arr1_pieces = [arr1[:h//2, :w//2], arr1[:h//2, w//2:], arr1[h//2:, :w//2], arr1[h//2:, w//2:]]
    arr2_pieces = [arr2[:h//2, :w//2], arr2[:h//2, w//2:], arr2[h//2:, :w//2], arr2[h//2:, w//2:]]
    arr3_pieces = [arr3[:h//2, :w//2], arr3[:h//2, w//2:], arr3[h//2:, :w//2], arr3[h//2:, w//2:]]
    arr4_pieces = [arr4[:h//2, :w//2], arr4[:h//2, w//2:], arr4[h//2:, :w//2], arr4[h//2:, w//2:]]

    # Randomly shuffle the pieces
    np.random.shuffle(arr1_pieces)
    np.random.shuffle(arr2_pieces)
    np.random.shuffle(arr3_pieces)
    np.random.shuffle(arr4_pieces)
    np.array(np.concatenate((arr1_pieces[0], arr2_pieces[1]),axis=0))

    # Concatenate the shuffled pieces to create new images
    arr_new1 = np.concatenate(
        (np.concatenate((arr1_pieces[0], arr2_pieces[1]),axis=0), 
         np.concatenate((arr3_pieces[2], arr4_pieces[3]), axis=0)),
     axis=1)
    
    arr_new2 = np.concatenate((
        np.concatenate((arr2_pieces[0], arr3_pieces[1]),axis=0),
        np.concatenate((arr4_pieces[2], arr1_pieces[3]), axis=0)),
        axis=1)
    
    arr_new3 = np.concatenate((
        np.concatenate((arr3_pieces[0], arr4_pieces[1]),axis=0),
        np.concatenate((arr1_pieces[2], arr2_pieces[3]), axis=0)),
        axis=1)
    
    arr_new4 = np.concatenate((
        np.concatenate((arr4_pieces[0], arr1_pieces[1]),axis=0),
        np.concatenate((arr2_pieces[2], arr3_pieces[3]), axis=0)),
        axis=1)                             
                               
    # Return the new images
    return arr_new1, arr_new2, arr_new3, arr_new4

In [ ]:
(x_train, _), (x_test, _) = cifar10.load_data()

In [ ]:
train_images_shuffled = []
train_labels_shuffled = []
for i in range(1400):
  number_of_images = 4
  # Randomly select 100 elements from the array
  random_selection1 = x_train[np.random.choice(x_train.shape[0], size=number_of_images, replace=False)]
  new_im1, new_im2, new_im3, new_im4 = shuffle_image_pieces(random_selection1[0],
                       random_selection1[1],
                       random_selection1[2],
                       random_selection1[3])
  
  shuffled = [new_im1, new_im2, new_im3, new_im4]

  train_images_shuffled = train_images_shuffled + shuffled
  train_labels_shuffled = train_labels_shuffled + [[10],[10],[10],[10]]

  # plt.figure(figsize=(10, 4))
  # for i in range(4):
  #     # Original image
  #     plt.subplot(2, 4, i + 1)
  #     plt.imshow(random_selection1[i])
  #     plt.axis('off')

      
  #     # Encoded image
  #     plt.subplot(2, 4, i + 5)
  #     plt.imshow(shuffled[i])
  #     plt.axis('off')

  # plt.show()

train_labels_shuffled = np.asarray(train_labels_shuffled)
train_images_shuffled = np.asarray(train_images_shuffled).astype('float32') / 255.

## USE AE

In [ ]:
# Load the saved model
encoder = tf.keras.models.load_model('/content/drive/MyDrive/ECE720/CIFAR_AE/encoder')
decoder = tf.keras.models.load_model('/content/drive/MyDrive/ECE720/CIFAR_AE/decoder')


# Plot the encoder output for the first 10 test images
number_of_images = 8000
# Load CIFAR-10 dataset
(x_train, _), (x_test, _) = cifar10.load_data()
# Randomly select 100 elements from the array
random_selection1 = x_train[np.random.choice(x_train.shape[0], size=number_of_images, replace=False)].astype('float32') / 255.
random_selection2 = x_train[np.random.choice(x_train.shape[0], size=number_of_images, replace=False)].astype('float32') / 255.

encoded_imgs = encoder.predict((random_selection1 + random_selection2)/2)
decoded_imgs = decoder.predict(encoded_imgs)
# decoded_imgs = autoencoder.predict(x_test[:10])
# plt.figure(figsize=(20, 4))
# for i in range(10):
#     # Original image
#     plt.subplot(2, 10, i + 1)
#     plt.imshow(x_test[i])
#     plt.axis('off')

    
#     # Encoded image
#     plt.subplot(2, 10, i + 11)
#     plt.imshow(decoded_imgs[i])
#     plt.axis('off')

# plt.show()


250/250 [==============================] - 0s 2ms/step


## Load images from cifar100 and add as oos

In [ ]:
class_names_cifar100 = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup',
    'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house',
    'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man',
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid',
    'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy',
    'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea',
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel',
    'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger', 'tractor',
    'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
    'worm'
]

count_oos = 500


# Load the CIFAR-10 and CIFAR-100 datasets
(_, _), (x_cifar10, y_cifar10) = cifar10.load_data()
(_, _), (x_cifar100, y_cifar100) = cifar100.load_data()

# Define the CIFAR-10 and CIFAR-100 class dictionaries
cifar10_classes = {
    0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer',
    5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'
}

cifar100_classes = {
    i: class_name for i, class_name in enumerate(class_names_cifar100)
}

# Get the list of classes that are unique to CIFAR-100
cifar100_unique_classes = set(cifar100_classes.values()) - set(cifar10_classes.values())

cifar100_unique_classes_keys = []
for key, value in cifar100_classes.items():
  if value in cifar100_unique_classes:
    cifar100_unique_classes_keys.append(key)

# Select n random images from CIFAR-100 that belong to the unique classes
indices = np.where(np.isin(y_cifar100, cifar100_unique_classes_keys))[0]
selected_indices = np.random.choice(indices, size=count_oos, replace=False)
x_selected_oos = x_cifar100[selected_indices].astype('float32') / 255.
y_selected_oos = y_cifar100[selected_indices]

# # Print the class labels of the selected images
# for label in y_selected_oos:
#     print(cifar100_classes[label[0]])


y_selected_oos.fill(10)
y_selected_oos_changed = np.random.randint(0, 10, size=(count_oos,1))

169001437/169001437 [==============================] - 13s 0us/step


# Final data prepration

In [ ]:
train_images = train_images.astype('float32') / 255.

train_labels_changed = copy.deepcopy(train_labels)

# Select 100 random images
random_indices = np.random.choice(train_labels_changed.shape[0], size=200, replace=False)

# Flip the labels of the selected images
train_labels_changed[random_indices] = np.random.randint(0, 10, size=(200,1))

In [ ]:
# # base without oos class
# x_all = np.concatenate([x_selected_oos, train_images], axis=0)
# y_all_changed = np.concatenate([y_selected_oos_changed, train_labels_changed], axis=0)
# y_all = np.concatenate([y_selected_oos, train_labels], axis=0)

In [ ]:
# # base with oos class
# x_all = np.concatenate([x_selected_oos, train_images, decoded_imgs, train_images_shuffled], axis=0)
# y_all_changed = np.concatenate([y_selected_oos_changed, 
#                                 train_labels_changed, 
#                                 np.full((len(decoded_imgs), 1), 10),
#                                 train_labels_shuffled], 
#                                 axis=0)
# y_all = np.concatenate([y_selected_oos, 
#                         train_labels, 
#                         np.full((len(decoded_imgs), 1), 10),
#                         train_labels_shuffled], 
#                         axis=0)

file_path = '/content/drive/MyDrive/ECE720/CIFAR_AE/Cifar_data/'
# Load the arrays from file
x_all_loaded = np.load(file_path + 'x_all.npy')
y_all_loaded = np.load(file_path + 'y_all.npy')
y_all_changed_loaded = np.load(file_path + 'y_all_changed.npy')

# base with oos class
x_all = np.concatenate([x_all_loaded, decoded_imgs, train_images_shuffled], axis=0)
y_all_changed = np.concatenate([y_all_changed_loaded,
                                np.full((len(decoded_imgs), 1), 10),
                                train_labels_shuffled], 
                                axis=0)
y_all = np.concatenate([y_all_loaded,
                        np.full((len(decoded_imgs), 1), 10),
                        train_labels_shuffled], 
                        axis=0)

In [ ]:
permutation = np.random.permutation(x_all.shape[0])
x_all = x_all[permutation]
y_all_changed = y_all_changed[permutation]
y_all = y_all[permutation]

# Training Classifier For Out of sample Prediction

In [ ]:
from sklearn.model_selection import KFold

# Assuming the dataframe is named "final_data" and you want to divide it into 5 folds
k = 5
kf = KFold(n_splits=k)

fold_indices = []
for train_index, test_index in kf.split(x_all):
    fold_indices.append((train_index, test_index))

In [ ]:
predctions = [[]] * len(x_all)
for i, (train_index, test_index) in enumerate(fold_indices):

  train_data = x_all[train_index]
  test_data = x_all[test_index]

  train_label = y_all_changed[train_index]
  test_label = y_all_changed[test_index]


  num_classes = 11
  y_train = keras.utils.to_categorical(train_label, num_classes)
  y_test = keras.utils.to_categorical(test_label, num_classes)

  x_train = train_data.astype('float32')
  x_test = test_data.astype('float32')


  # loading B0 pre-trained on ImageNet without final aka fiature extractor
  model = enet.EfficientNetB0(include_top=False, input_shape=(32,32,3), pooling='avg', weights='imagenet')

  # building 2 fully connected layer 
  x = model.output

  x = BatchNormalization()(x)
  x = Dropout(0.7)(x)

  x = Dense(512)(x)
  x = BatchNormalization()(x)
  x = Activation(swish_act)(x)
  x = Dropout(0.5)(x)

  x = Dense(128)(x)
  x = BatchNormalization()(x)
  x = Activation(swish_act)(x)

  # output layer
  predictions = Dense(num_classes, activation="softmax")(x)
  model_final = Model(inputs = model.input, outputs = predictions)


  # model compilation
  model_final.compile(loss='categorical_crossentropy',
                optimizer=Adam(0.0001),
                metrics=['accuracy'])

  # mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_acc')
  reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1,)

  #print("Training....")
  model_final.fit(x_train, y_train,
                  batch_size=512,
                  epochs=15,
                  validation_data=(x_test, y_test),
                  callbacks=[reduce_lr],
                  shuffle=True,
                  verbose=1)


  pred_probs = model_final.predict(x_test)
  for j in range(len(test_index)):
    predctions[test_index[j]] = pred_probs[j]

In [ ]:
preds = [np.argmax(i) for i in predctions]
labels = y_all_changed

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, precision_score, recall_score

In [ ]:
cm  = confusion_matrix(labels, preds)
plt.figure()
plot_confusion_matrix(cm, figsize=(16,12), hide_ticks=True, cmap=plt.cm.Blues)
plt.xticks(range(num_classes), list(range(num_classes)), fontsize=12)
plt.yticks(range(num_classes),  list(range(num_classes)), fontsize=12)
plt.show()

# Confident Learning Step

In [ ]:
# STEP 1 - Compute confident joint
import cleanlab

# Verify inputs
labels = y_all_changed.reshape((1,-1))[0]
pred_probs = np.asarray(predctions)

# Find the number of unique classes if K is not given
K = len(np.unique(labels))

# Estimate the probability thresholds for confident counting
# You can specify these thresholds yourself if you want
# as you may want to optimize them using a validation set.
# By default (and provably so) they are set to the average class prob.
thresholds = [np.mean(pred_probs[:, k][labels == k]) for k in range(K)]  # P(label^=k|label=k)
thresholds = np.asarray(thresholds)

# Compute confident joint
confident_joint = np.zeros((K, K), dtype=int)
for i, row in enumerate(pred_probs):
    # row = np.array(row)
    s_label = labels[i]
    # Find out how many classes each example is confidently labeled as
    confident_bins = row >= thresholds - 1e-6
    num_confident_bins = sum(confident_bins)
    # If more than one conf class, inc the count of the max prob class
    if num_confident_bins == 1:
        confident_joint[s_label][np.argmax(confident_bins)] += 1
    elif num_confident_bins > 1:
        confident_joint[s_label][np.argmax(row)] += 1

# Normalize confident joint (use cleanlab, trust me on this)
confident_joint = cleanlab.count.calibrate_confident_joint(confident_joint, labels)

cleanlab.internal.util.print_joint_matrix(confident_joint)


In [ ]:
# STEP 2 - Find label errors

# We arbitrarily choose at least 5 examples left in every class.
# Regardless of whether some of them might be label errors.
MIN_NUM_PER_CLASS = 5
# Leave at least MIN_NUM_PER_CLASS examples per class.
# NOTE prune_count_matrix is transposed (relative to confident_joint)
prune_count_matrix = cleanlab.filter._keep_at_least_n_per_class(
    prune_count_matrix=confident_joint.T,
    n=MIN_NUM_PER_CLASS,
)

s_counts = np.bincount(labels)
noise_masks_per_class = []
# For each row in the transposed confident joint
for k in range(K):
    noise_mask = np.zeros(len(pred_probs), dtype=bool)
    pred_probs_k = pred_probs[:, k]
    if s_counts[k] > MIN_NUM_PER_CLASS:  # Don't prune if not MIN_NUM_PER_CLASS
        for j in range(K):  # noisy label index (k is the true label index)
            if k != j:  # Only prune for noise rates, not diagonal entries
                num2prune = prune_count_matrix[k][j]
                if num2prune > 0:
                    # num2prune'th largest p(classk) - p(class j)
                    # for x with noisy label j
                    margin = pred_probs_k - pred_probs[:, j]
                    s_filter = labels == j
                    threshold = -np.partition(-margin[s_filter], num2prune - 1)[
                        num2prune - 1
                    ]
                    noise_mask = noise_mask | (s_filter & (margin >= threshold))
        noise_masks_per_class.append(noise_mask)
    else:
        noise_masks_per_class.append(np.zeros(len(labels), dtype=bool))

# Boolean label error mask
label_errors_bool = np.stack(noise_masks_per_class).any(axis=0)

# Remove label errors if given label == model prediction
for i, pred_label in enumerate(pred_probs.argmax(axis=1)):
    # np.all lets this work for multi_label and single label
    if label_errors_bool[i] and np.all(pred_label == labels[i]):
        label_errors_bool[i] = False

# Convert boolean mask to an ordered list of indices for label errors
label_errors_idx = np.arange(len(labels))[label_errors_bool]
# self confidence is the holdout probability that an example
# belongs to its given class label
self_confidence = np.array([np.mean(pred_probs[i][labels[i]]) for i in label_errors_idx])
margin = self_confidence - pred_probs[label_errors_bool].max(axis=1)
label_errors_idx = label_errors_idx[np.argsort(margin)]

print("Indices of label errors found by confident learning:")
print("Note label errors are sorted by likelihood of being an error")
print("but here we just sort them by index for comparison with above.")
print(np.array(sorted(label_errors_idx)))


Indices of label errors found by confident learning:
Note label errors are sorted by likelihood of being an error
but here we just sort them by index for comparison with above.
[    0     1     2 ... 66597 66598 66599]


In [ ]:
def find_differences(arr1, arr2):
    """Returns the indexes where the values in the two arrays differ"""
    diffs = []
    for i in range(len(arr1)):
        if arr1[i] != arr2[i]:
            diffs.append(i)
    return diffs

actual_label_errors = find_differences(y_all,y_all_changed)
NUM_ERRORS= len(actual_label_errors)
NUM_ERRORS

In [ ]:
label_errors_idx_pruned = []
for index in label_errors_idx:
  if y_all[index] !=10 and y_all_changed[index] !=10:
    label_errors_idx_pruned.append(index)


In [ ]:
score = (sum([e in label_errors_idx for e in actual_label_errors]) / NUM_ERRORS) 
print("% actual errors that confident learning found: {:.0%}".format(score))
# score = sum([e in actual_label_errors for e in label_errors_idx]) / len(label_errors_idx)